In [ ]:
import numpy as np
import torch
import tensorflow as tf
import scipy.sparse as sp
import time
import matplotlib.pyplot as plt

# Generate matrix sizes
matrix_sizes = []
matrix_sizes.extend(range(3, 100, 5))  # 3 to 100
matrix_sizes.extend(range(100, 1000, 50))  # 100 to 1000
matrix_sizes.extend(range(1000, 10000, 500))  # 1000 to 10000
matrix_sizes.extend(range(10000, 15001, 5000))  # 10000 to 15000

libraries = ['numpy', 'pytorch', 'tensorflow']
matrix_type = 'sparse'

# Dictionary to store results
results = {lib: {'sizes': [], 'times': []} for lib in libraries}

In [ ]:
matrix_type = 'dense'
# matrix_type = 'sparse'

In [ ]:
# Perform SVD comparison
for size in matrix_sizes:
    print(f"\nProcessing matrix size: {size}x{size}")

    # Generate matrix
    if matrix_type == 'dense':
        matrix = np.random.rand(size, size)
    else:  # sparse
        matrix = np.random.rand(size, size)
        mask = np.random.random(matrix.shape) < 0.95
        matrix[mask] = 0
        matrix = sp.csr_matrix(matrix)

    for lib in libraries:
        start_time = time.time()

        try:
            if lib == 'numpy':
                if matrix_type == 'dense':
                    U, S, Vt = np.linalg.svd(matrix, full_matrices=False)
                else:  # sparse
                    U, S, Vt = sp.linalg.svds(matrix, k=min(matrix.shape) - 1)
                    U = np.pad(U, ((0, 0), (0, matrix.shape[0] - U.shape[1])), mode='constant')
                    Vt = np.pad(Vt, ((0, matrix.shape[1] - Vt.shape[0]), (0, 0)), mode='constant')

            elif lib == 'pytorch':
                if matrix_type == 'dense':
                    U, S, Vt = torch.linalg.svd(torch.tensor(matrix), full_matrices=False)
                else:  # sparse
                    matrix_dense = matrix.toarray()
                    U, S, Vt = torch.linalg.svd(torch.tensor(matrix_dense), full_matrices=False)
                U, S, Vt = U.numpy(), S.numpy(), Vt.numpy()

            elif lib == 'tensorflow':
                if matrix_type == 'dense':
                    S, U, Vt = tf.linalg.svd(matrix, full_matrices=False)
                else:  # sparse
                    matrix_dense = matrix.toarray()
                    S, U, Vt = tf.linalg.svd(matrix_dense, full_matrices=False)
                U, S, Vt = U.numpy(), S.numpy(), Vt.numpy()

            else:
                raise ValueError(f"Unsupported library: {lib}")

            end_time = time.time()

            results[lib]['sizes'].append(size)
            results[lib]['times'].append(end_time - start_time)

            print(f"{lib.upper()} SVD completed in {end_time - start_time:.4f} seconds")

        except Exception as e:
            print(f"{lib.upper()} Error: {str(e)}")
            results[lib]['sizes'].append(size)
            results[lib]['times'].append(np.nan)

In [ ]:
# Plot results
plt.figure(figsize=(15, 10))
for lib in libraries:
    plt.plot(results[lib]['sizes'], results[lib]['times'], label=f"{lib.upper()} SVD", marker='o')

plt.title('SVD Performance Comparison on Sparse Matrices', fontsize=16)
plt.xlabel('Matrix Size', fontsize=14)
plt.ylabel('Time (seconds)', fontsize=14)
plt.legend(fontsize=12)
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which="both", ls="-", alpha=0.2)
plt.tight_layout()
plt.savefig('svd_sparse_performance_comparison.png')
plt.show()

In [ ]:
# Plot results
plt.figure(figsize=(15, 10))
for lib in libraries:
    plt.plot(results[lib]['sizes'], results[lib]['times'], label=f"{lib.upper()} SVD", marker='o')

plt.title('SVD Performance Comparison on Dense Matrices', fontsize=16)
plt.xlabel('Matrix Size', fontsize=14)
plt.ylabel('Time (seconds)', fontsize=14)
plt.legend(fontsize=12)
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which="both", ls="-", alpha=0.2)
plt.tight_layout()
plt.savefig('svd_dense_performance_comparison.png')
plt.show()